In [200]:
from datetime import date,timedelta 
import math
import pandas as pd
import numpy as np
import pyodbc
import mysql.connector
import sqlalchemy
from sqlalchemy import create_engine
server = 'bieno-da-d-80166-unilevercom-sql-01.database.windows.net'
database = 'bieno-da-d-80166-unilevercom-sqldb-01'
username = 'PROD_LoadPool'
password = 'Dlk45^8&&dodsslk12Ad7*as'
driver = 'SQL Server Native Client 11.0'
port = '1433'
pd.options.mode.chained_assignment = None
data_directory = 'C:/Users/tristan.dresbach/OneDrive - Unilever/'

In [201]:
#get fullfilled orders:
sql ='''
select MaterialNumber,SalesOrderCode,CustomerSoldTo,ReportingDate, QuantityExpected,QuantityDispatched from RawReporting_BDL.FactSalesOrderLine
where ReportingDate >= '2021-03-01' and ReportingDate < '2021-03-28' and QuantityOrdered = QuantityDispatched and QuantityOrdered <> 0 and SalesOrganisationCode in ('0001')
'''
#EDIT BACK SALES ORGS: '0002','0003','0007'
connection_string = 'DRIVER={};SERVER={};PORT={};DATABASE={};UID={};PWD={}'.format(
    driver, server, port, database, username, password)
cnxn = pyodbc.connect(connection_string)
good_orders = pd.read_sql(sql, cnxn)  # Can be assigned to any variable name
cnxn.close()

In [202]:
#final customer expected order qty
# if we want to run the code for a specific list of input DUs given to us by the business, here: "Book1.xlsx",
#we filter our data for those items
#tgt_items = pd.read_excel('Book1.xlsx')
#tgt_items['DU'] = [(18-len(str(x)))*'0' + str(x) for x in tgt_items['DU']]
#good_orders = good_orders[good_orders['MaterialNumber'].isin(tgt_items['DU'].tolist())]
#good_orders = good_orders.reset_index(drop=True)

In [203]:
#create our cuts column to be used later on
good_orders['LBD_QT'] = good_orders['QuantityExpected'] - good_orders['QuantityDispatched']

In [204]:
#get cut orders:
#the cut column is "(LI.QuantityLossBeforeDispatch*-1) [LBD_QT]", we use the same date range as for the previous "good_orders" SQL query.
# last line of SQL query: "AND LI.QuantityLossBeforeDispatch <> 0" guarantees that we are only pulling cuts

sql ='''
    SELECT
            LI.MaterialNumber
            , LI.[SalesOrderCode]
            , LI.CustomerSoldTo
            , LI.ReportingDate
            , LI.QuantityExpected
            ,LI.QuantityDispatched

           
           ,(LI.QuantityLossBeforeDispatch*-1) [LBD_QT]
           
        FROM [RawReporting_BDL].[FactSalesOrderLine] LI
        INNER JOIN [TeradataConnect].[DimCalendar] CAL ON LI.ReportingDate = CAL.Date_Dt
        INNER JOIN [RawReporting_BDL].[DimSalesOrganisation] AS SO ON LI.SalesOrganisationCode =SO.SalesOrganisationCode
        INNER JOIN [RawReporting_BDL].[DimDistributionChannel] AS DC ON LI.DistributionChannelCode =DC.DistributionChannelCode
        INNER JOIN [RawReporting_BDL].[HierarchyCordilleraLocalProduct] AS PS ON LI.MaterialNumber =PS.MaterialNumber AND LI.SalesOrganisationCode=PS.SalesOrganisationCode AND LI.DistributionChannelCode=PS.DistributionChannel
        WHERE CAL.Days_From_Current > 0 and CAL.Date_Dt >= '2021-03-01' and CAL.Date_dt < '2021-03-28'
        AND LI.DistributionChannelCode IN ('10','61') ---different distribution channels, 
        AND LI.SalesOrganisationCode IN ('0001')
        AND LI.UnitOfMeasureCode = 'CS'
        AND PS.LocalLevel3Code NOT IN ( '006', '0FF', '008', '04D', '0F1')
        AND LI.QuantityLossBeforeDispatch <> 0
'''

#'0002','0003','0007'
connection_string = 'DRIVER={};SERVER={};PORT={};DATABASE={};UID={};PWD={}'.format(
    driver, server, port, database, username, password)
cnxn = pyodbc.connect(connection_string)
orders_cut = pd.read_sql(sql, cnxn)  # Can be assigned to any variable name
cnxn.close()
#AND LI.QuantityLossBeforeDispatch <> 0

In [205]:
#these two lines below are part of the same process to filter the data for the items in which we are interested from that input file

#orders_cut = orders_cut[orders_cut['MaterialNumber'].isin(tgt_items['DU'].tolist())]
#orders_cut = orders_cut.reset_index(drop=True)

In [206]:
#get our order df which will serves as our data base
orders = pd.concat([good_orders,orders_cut],ignore_index=True)
orders.rename(columns={'LBD_QT':'Cuts','ReportingDate':'date'},inplace=True)
list_DUs = list(set(orders['MaterialNumber']))

In [208]:
#list of sold tos, for names:
customers = list(set(orders['CustomerSoldTo']))
customers_str = str(customers)

In [209]:
#get all customer names, using the list of soldtos we just generated from our data:
sql ='''
select CustomerCode,EccCustomerLevel5Name from RawReporting_BDL.HierarchyCordilleraCustomer
where CustomerCode in ''' + customers_str.replace('[','(').replace(']',')')
connection_string = 'DRIVER={};SERVER={};PORT={};DATABASE={};UID={};PWD={}'.format(
    driver, server, port, database, username, password)
cnxn = pyodbc.connect(connection_string)
cus_names = pd.read_sql(sql, cnxn)  # Can be assigned to any variable name
cnxn.close()

In [210]:
#append customer names and merge our base df ("orders") with the customer related data:
cus_names.rename(columns={'CustomerCode':'CustomerSoldTo','EccCustomerLevel5Name':'Customer'},inplace=True)
cus_names = cus_names.drop_duplicates(subset=['CustomerSoldTo'],keep='first').reset_index(drop=True)
df = pd.merge(orders,cus_names,how='left',on='CustomerSoldTo')

In [211]:
#This is an example of subsetting for a specific customer: Walmart only
#df = df[df['Customer'] == 'WAL-MART']

In [213]:
#prep df for analysis:
#we filter out weird data issues like DUs that have empty spaces, weird amount of characters, are = -1
#create key column for future looping as well
df_f = df.copy()
#df_f = df_f.sort_values(['MaterialNumber','date']).reset_index(drop=True)
df_f = df_f[~df_f['MaterialNumber'].str.contains(' ')].reset_index(drop=True)
df_f = df_f[~df_f['MaterialNumber'].str.len() < 5].reset_index(drop=True)
#df_f = df_f[df_f['date'] <= date.today()]
#df_f = df_f[(df_f['QuantityDispatched'] >= 0) & (df_f['QuantityOriginal'] > 0)].reset_index(drop=True)
df_f = df_f[df_f['MaterialNumber'] != '-1']
df_f['key'] = df_f['MaterialNumber'] + '_' + df_f['CustomerSoldTo']

In [215]:
#setup output:
#want to view the data at a DU & lane level hence we do a groupby to know how much should have shipped and wasn't
#we create a list of keys from this that will be the DU/lane we are interested in looping through to identify true demand

output_df = df_f.groupby(['MaterialNumber','CustomerSoldTo','Customer','key'],as_index=False)[['QuantityExpected','Cuts']].sum()
output_df.drop_duplicates(inplace=True)
output_df = output_df[output_df['Cuts'] > 1]
output_df = output_df[output_df['QuantityExpected'] > 1]
output_df = output_df.sort_values(['Cuts'],ascending=False)
output_df = output_df.reset_index(drop=True)
df_f_m = df_f[df_f['key'].isin(list(output_df['key']))]
df_f_m = df_f_m.reset_index(drop=True)

In [1]:

loss_output_nocap = []
key_output_nocap = []
date_output_nocap = []
avg_dates_nocap = []
ratios = []
days_left_t = []
next_p_t = []

#iterate through each key:
for key in list(output_df['key']):
    #to get an idea of timing, prints the % completed as it goes through
    if len(key_output_nocap) % 300 == 0:
        print('% done :',round(len(key_output_nocap)/len(output_df),2))
    #subset the df for POs we are specifically interested in:
    df = df_f_m[df_f_m['key'] == key]
    df = df[df['QuantityExpected'] > 0]
    cuts_t = []
    df = df.sort_values(by=['date'])
    dates_l = sorted(set(df['date']))
    ordered_t= []
    cuts_q = []
    ordered_q = []
    leng = (date.today() - min(list(df['date']))).days
    #create our base lists of day differences, cuts and orders
    full_cuts = []
    for i in range(leng+1):
        day =  date.today() - timedelta(days=i)
        cut = sum(list(df[df['date'] == day]['Cuts']))
        ord_ = sum(list(df[df['date'] == day]['QuantityExpected']))
        #print(day,ord_)
        if ord_ > 0.1:
            cuts_q += [cut]
            ordered_q += [ord_]
        full_cuts += [cut]
        cuts_t+= [cut]
        ordered_t += [ord_]
    #re verse the lists so they are in chrono-logical order
    full_cuts = full_cuts[::-1]
    cuts_t = cuts_t[::-1]
    ordered_t = ordered_t[::-1]
    cuts_q = cuts_q[::-1]
    ordered_q = ordered_q[::-1]
    #get the average orders and cuts:
    avg_order = np.mean([x for x in cuts_q if x > 1])
    avg_cuts = np.mean([x for x in ordered_q if x > 1])
    
    count_indx = []
    ct = 0
    for x in ordered_t:
        if x > 0.1:
            count_indx += [ct]
        ct +=1
    
    #initialize variables in the below loop:
    i = 0
    loop = True
    cumu = 0
    indx_cumu = []
    temp_loss = 0
    temp_dates = []
    date_avgs = []
    loss_list = []
    ordered_list = []
    drop_indxs = []
    full_indxs = []
    full_loss = []
    min_date = min(list(df['date']))
    
    #we first check if there are cuts, and if so get a sum for each subsequent cut quantity, as a roling sum:
    while loop == True:
        temp_date = []
        temp_avg = []
        cc1 = 0
        cc2 = 0
        cc3 = 0
        loss = 0
        if i+2 < len(cuts_q):
            cc1 += cuts_q[i] + cuts_q[i+1]
        if i+3 < len(cuts_q):
            cc2 += cc1 + cuts_q[i+2]
        if i+4 < len(cuts_q):
            cc3 += cc2 + cuts_q[i+3]
        
        #use these subsequent cuts to test if: we have an increase in cuts, if the next cut is above average cuts and if the next order is above 0.3* average order and if the sum of cuts is above the average cuts.
        #we also check to make sure all these cuts/orders happen within the window of 11 days:
        
        if (i+1 < len(cuts_q)) and (cuts_q[i] < cuts_q[i+1]) and (cuts_q[i+1]>=1.0*avg_cuts) and (ordered_q[i+1] >= 0.3*avg_order) and (cuts_q[i] >= 0.1*avg_cuts) and ( (count_indx[i+1] - count_indx[i]) < 11) :
            temp_loss += cuts_q[i+1] + cuts_q[i]
            temp_date = [min_date + timedelta(days = count_indx[i]),min_date + timedelta(days = count_indx[i+1])]
            #print(cuts_q[i+1]-cuts_q[i],'  c1') 
            i += 1
            
        elif  ((i+2 < len(cuts_q)) and (cc1 < cuts_q[i+2]) and (cuts_q[i+2]>=1.0*avg_cuts) and (ordered_q[i+2] >= 0.3*avg_order) and (cc1 >= 0.1*avg_cuts) and  (count_indx[i+2] - count_indx[i] < 11)):
            #print(cuts_q[i+2] - cc1,'  c2')
            temp_loss += cuts_q[i+2] + cc1
            temp_date = [min_date + timedelta(days = count_indx[i]),min_date + timedelta(days = count_indx[i+1]),min_date + timedelta(days = count_indx[i+2])]
            i += 2
            
        elif  ((i+3 < len(cuts_q)) and (cc2 < cuts_q[i+3]) and (cuts_q[i+3]>=1.0*avg_cuts) and (ordered_q[i+3] >= 0.3*avg_order) and (cc2 >= 0.1*avg_cuts) and (count_indx[i+3] - count_indx[i] < 11)):
            #print(cuts_q[i+3] - cc2,'  c2')
            temp_loss += cuts_q[i+3] + cc2
            temp_date = [min_date + timedelta(days = count_indx[i]),min_date + timedelta(days = count_indx[i+1]),min_date + timedelta(days = count_indx[i+2]),min_date + timedelta(days = count_indx[i+3])]
            i += 3

        elif  ((i+4 < len(cuts_q)) and (cc3 < cuts_q[i+4]) and (cuts_q[i+4]>=1.0*avg_cuts) and (ordered_q[i+4] >= 0.3*avg_order) and (cc3 >= 0.1*avg_cuts) and (count_indx[i+4] - count_indx[i] < 11)):
            #print(cuts_q[i+4] - cc3,'  c3')
            temp_loss += cuts_q[i+4] + cc3
            temp_date = [min_date + timedelta(days = count_indx[i]),min_date + timedelta(days = count_indx[i+1]),min_date + timedelta(days = count_indx[i+2]),min_date + timedelta(days = count_indx[i+3]),min_date + timedelta(days = count_indx[i+4])]
            i += 4

        else:
            i += 1

        if i > len(cuts_q):
            loop = False
        #for the above patterns that have been flagged as True, calcualte the amount of losses (for orders and cuts) that took place and what time frame it was on
        if len(temp_date) != 0:
            temp_dates += temp_date
            avg_date = min(temp_date) + timedelta(days = round((max(temp_date) - min(temp_date)).days/2,0))
            date_avgs += [avg_date]
            #setup lists:
            indxs = []
            for day in temp_date:
                indxs += [dates_l.index(day)]
            ordered_loss = []
            ordered_q_sum = []
            drop_indxs += indxs
            full_indxs += [indxs] 
            for i in indxs:
                ordered_loss += [cuts_q[i]]
                ordered_q_sum += [ordered_q[i]]
            full_loss += [ordered_loss]
            loss_list +=[sum(ordered_loss)]
            ordered_list += [sum(ordered_q_sum)]
    
    #this adjusts the losses over time, want to reflect the importance of recent losses vs older losses
    if temp_loss != 0:
        cuts_base = [i for j, i in enumerate(cuts_q) if j not in drop_indxs]
        dates_base = [i for j, i in enumerate(dates_l) if j not in drop_indxs]
        #print(loss_list,ordered_list,full_indxs)
        ordered_study =  ordered_q
        loss_study = [cuts_q[i] if i in drop_indxs else 0 for i in range(len(ordered_study))]
        ordered_adj = ordered_study
        loss_adj = loss_study
        
        adj_coeffs = [round(math.exp(-i/len(loss_adj)),2) for i in range(len(loss_adj))][::-1]
        adj_coeffs = [round(x*x,2) for x in adj_coeffs]
        ordered_adj_coef = [round(x*y,2) for x,y in zip(ordered_adj,adj_coeffs)]
        loss_adj_coef = [round(x*y,2) for x,y in zip(loss_adj,adj_coeffs)]
        golden_ratio = round(sum(loss_adj_coef)/sum(ordered_adj_coef),2)
        ratios += [golden_ratio]
        
        date_list = [min(dates_l) + datetime.timedelta(days=i) for i in range(leng+1)]

        ratios += ['no loss']
            
    
    key_output_nocap += [key]
    date_output_nocap += [temp_dates]
    avg_dates_nocap += [date_avgs]
    
    #remove duplicate counts:
    days_sub = {}
    for day in list(set(temp_dates)):
        ct_ = temp_dates.count(day)
        if ct_ > 1:
            temp_loss -= (ct_-1) * list(df[df['date'] == day]['Cuts'])[0]
        
    loss_output_nocap += [temp_loss]

NameError: name 'output_df' is not defined

In [ ]:
#get data for allocation script:
ratios_df = pd.DataFrame()
data_directory = 'C:/Users/tristan.dresbach/OneDrive - Unilever/CC_orders/'
ratios_df['key'] = key_output_nocap[0:len(key_output_nocap)]
ratios_df['loss'] = loss_output_nocap[0:len(key_output_nocap)]
ratios_df['date'] = date_output_nocap[0:len(key_output_nocap)]
ratios_df['ratio'] = ratios[0:len(key_output_nocap)]
outpath = data_directory + 'ratio_data.xlsx'
ratios_df.to_excel(outpath)

In [222]:
#in-depth output for the business:


#create our data frame with the data from the previous loops
no_cap = pd.DataFrame()
no_cap['key'] = key_output_nocap[0:len(key_output_nocap)]
no_cap['loss'] = loss_output_nocap[0:len(key_output_nocap)]
no_cap['date'] = date_output_nocap[0:len(key_output_nocap)]
no_cap['ratio'] = ratios[0:len(key_output_nocap)]

final_nocap = pd.merge(output_df,no_cap,how='left',on='key')

final_nocap = final_nocap.fillna('not ran')

study_nocap = final_nocap.copy()

#the below is just getting data the business likes to see from SQL and diong a bunch of merges
#get DU rankings:
sql ='''
select MaterialNumber,ABCD_NSV as priority from Segmentation.MasterAbcdByDu
where MaterialNumber in ''' + str(list(study_nocap['MaterialNumber'])).replace('[','(').replace(']',')')
connection_string = 'DRIVER={};SERVER={};PORT={};DATABASE={};UID={};PWD={}'.format(
    driver, server, port, database, username, password)
cnxn = pyodbc.connect(connection_string)
prio = pd.read_sql(sql, cnxn)  # Can be assigned to any variable name
cnxn.close()

study_nocap = pd.merge(study_nocap,prio,how='left',on='MaterialNumber')

#get item names:
sql ='''
select MaterialNumber,MaterialDescription as Description from RawReporting_BDL.DimMaterial
where MaterialNumber in ''' + str(list(study_nocap['MaterialNumber'])).replace('[','(').replace(']',')')
connection_string = 'DRIVER={};SERVER={};PORT={};DATABASE={};UID={};PWD={}'.format(
    driver, server, port, database, username, password)
cnxn = pyodbc.connect(connection_string)
mat_desc = pd.read_sql(sql, cnxn)  # Can be assigned to any variable name
cnxn.close()

mat_desc = mat_desc.drop_duplicates(subset=['MaterialNumber'], keep='first').reset_index(drop=True)

study_nocap = pd.merge(study_nocap,mat_desc,how='left',on='MaterialNumber')


#get description and brand:
sql = ('''
select A.MaterialNumber,APOBrandName,CategoryName,DivisionName from RawReporting_BDL.HierarchyCordilleraCategory A
left join (select MaterialNumber,MaterialDescription,APOBrandName from RawReporting_BDL.HierarchyAPOCordilleraProduct) B
on A.MaterialNumber = B.MaterialNumber
where A.MaterialNumber in ''' + str(list(study_nocap['MaterialNumber'])).replace('[','(').replace(']',')')
)

connection_string = 'DRIVER={};SERVER={};PORT={};DATABASE={};UID={};PWD={}'.format(
    driver, server, port, database, username, password)
cnxn = pyodbc.connect(connection_string)
ref_df = pd.read_sql(sql, cnxn)  # Can be assigned to any variable name
ref_df = ref_df.drop_duplicates(subset=['MaterialNumber'], keep='first').reset_index(drop=True)


study_nocap = pd.merge(study_nocap,ref_df,how='left',on='MaterialNumber')



by_items_nc = study_nocap.groupby(['MaterialNumber','Description','APOBrandName','CategoryName','DivisionName','priority'],as_index=False)[['loss','QuantityExpected','Cuts']].sum()
by_items_nc = by_items_nc.sort_values(['loss'],ascending = False)
by_items_nc = by_items_nc.reset_index(drop=True)


by_customer_nc = study_nocap.groupby(['Customer'],as_index=False)[['loss','QuantityExpected','Cuts']].sum()
by_customer_nc = by_customer_nc.sort_values(['loss'],ascending=False)
by_customer_nc = by_customer_nc.reset_index(drop=True)

In [223]:
#tgt = study_nocap[study_nocap['Customer'] == 'TARGET']
#export = tgt.groupby(by=['MaterialNumber'],as_index=False).sum().sort_values(by=['loss'],ascending=False)
#export.to_excel('tgt cuts_all_items_c.xlsx')

In [224]:
by_items_nc['Cuts'] = [y if x > y else x for x,y in zip(by_items_nc['Cuts'],by_items_nc['QuantityExpected'])]

In [60]:
#study.name = 'study'
study_nocap.name = 'study_nocap'
#by_customer.name = 'by_customer'
by_customer_nc.name = 'by_customer_nc'
by_items_nc.name = 'by_items_nc'
#by_items.name = 'by_items'

In [225]:
date_mod = []
        
date_mod_nc = []
for x in list(study_nocap['date']):
    if x != 0:
        temp_date_mod_nc = []
        for y in x:
            try:
                z = y.strftime("%m/%d/%Y")
                temp_date_mod_nc += [z]
            except:
                pass
        date_mod_nc += [temp_date_mod_nc]
    else:
        date_mod_nc += ['not run']
        
        

In [226]:
#study['date_mod'] = date_mod
study_nocap['date_mod'] = date_mod_nc

In [63]:
#prep for full export:
df_f_m = df_f_m.reset_index(drop=True)

#df_list = [study,study_nocap,by_customer,by_customer_nc,by_items_nc,by_items]
df_list = [study_nocap,by_customer_nc,by_items_nc]

#study.name = 'study'
study_nocap.name = 'study_1.0'
#by_customer.name = 'by_customer'
by_customer_nc.name = 'by_customer_1.0'
by_items_nc.name = 'by_items_1.0'
#by_items.name = 'by_items'

writer = ExcelWriter('Repeated_Ordering_1.0_2_22_2020.xlsx')
for n, df in enumerate(df_list):
    df.to_excel(writer, str(df.name))
writer.save()

In [229]:
#study_nocap.to_excel('WMT_cut_fake_cuts.xlsx')